Adapted from https://github.com/astudillo20lca/hypercognitive

**Conference Notebook Kernel: `bw25`**

In [ ]:
import shutil
from pathlib import Path

import bw2data as bwd
import bw2calc as bwc
import bw2io as bwi

This notebook creates minimalistic versions of the foreground, biosphere and impact assessment methods databases as lightweight data packages.

In [ ]:
bwi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

In [ ]:
bwd.projects.set_current('ei38-teaching-25')

In [ ]:
assert 'biosphere3' in bwd.databases

minimal verison of the biosphere with only the nodes I need

In [ ]:
if 'mini_biosphere' in bwd.databases:

    print(f"deleting {bwd.Database('mini_biosphere').name}")
    del bwd.databases['mini_biosphere']

In [ ]:
co2 = next(x for x in bwd.Database("biosphere3") 
           if x['name'] == 'Carbon dioxide, fossil'
           and x['categories'] == ('air',))

ch4 = next(x for x in bwd.Database("biosphere3") 
           if x['name'] == 'Methane, fossil'
           and x['categories'] == ('air',))

In [ ]:
co2_dict = {k:v for k,v in co2.as_dict().items() if k not in ['id','code']}
ch4_dict = {k:v for k,v in ch4.as_dict().items() if k not in ['id','code']}

co2_dict['code'] = 'CO2'
ch4_dict['code'] = 'CH4'
co2_dict['database'] = 'mini_biosphere'
ch4_dict['database'] = 'mini_biosphere'

bio_db = bwd.Database("mini_biosphere")
bio_db.register()

co2_new = bio_db.new_activity(**co2_dict)
co2_new.save()

ch4_new = bio_db.new_activity(**ch4_dict)
ch4_new.save()

bio_db_dpgk = bio_db.datapackage()

minimal verion of impact assessment method

In [ ]:
# map between ef in the original biosphere database and the simplified one
ef_keys_dict = {
co2.key:co2_new.key,
ch4.key:ch4_new.key
}

ipcc2013 = bwd.Method(('IPCC 2013', 'climate change', 'GWP 100a'))
required_cfs = [(ef_keys_dict.get(key),cf) for key,cf in ipcc2013.load() if key in ef_keys_dict]
ipcc_simplified = bwd.Method(('IPCC','simple'))
ipcc_simplified.write(required_cfs)
assert ipcc_simplified.filepath_processed().is_file()

In [ ]:
ipcc_simplified.register()

In [ ]:
ipcc_simplified.metadata

example of foreground database

In [ ]:
a_key = ("testdb", "machine_a")

act_a_def = {
    'name': 'a',
    'unit': 'kilogram',
    'exchanges': [{"input": co2_new.key, "type": "biosphere", "amount": 10},
                  {"input": a_key, "output":a_key,'type':'production','amount':1},
                  {"input": ch4_new.key, "type": "biosphere", "amount": 1},
                 ],
    }

db = bwd.Database("testdb")
db.write(
    {
    a_key : act_a_def,
    }
    )

calculate via normal procedure

In [ ]:
lca = bwc.LCA({db.get('machine_a'):1},method=ipcc_simplified.name)

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
db.process(csv=True)

In [ ]:
bio_db.process(csv=True)

get the related datapackages

In [ ]:
shutil.copyfile(src=ipcc_simplified.filepath_processed(),
                dst=Path.cwd()/ipcc_simplified.filename_processed());

In [ ]:
shutil.copyfile(src=db.filepath_processed(),
                dst=Path.cwd()/db.filename_processed());

In [ ]:
shutil.copyfile(
src=bio_db.filepath_processed(),
dst=Path.cwd()/bio_db.filename_processed());